In [5]:
%load_ext autoreload
%autoreload 2

In [ ]:
!pip install tensorflow-gan
! pip install tensorflow-probability

In [6]:
# Generate graphs for demonstration purposes
import utils.graph_generators as gen
import torch
import dgl

grids = gen.make_grid_graphs()
lobsters = gen.make_lobster_graphs()

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
grids = [dgl.DGLGraph(g).to(device) for g in grids] # Convert graphs to DGL from NetworkX
lobsters = [dgl.DGLGraph(g).to(device) for g in lobsters] # Convert graphs to DGL from NetworkX

c:\Users\cs843\AppData\Local\Programs\Python\Python310\lib\site-packages\dgl\heterograph.py:72: DGLWarning: Recommend creating graphs by `dgl.graph(data)` instead of `dgl.DGLGraph(data)`.
  dgl_warning('Recommend creating graphs by `dgl.graph(data)`'


In [3]:
b=gen.load_proteins()

In [10]:
b[0].nodes(data=True)

NodeDataView({1: {'label': 4}, 2: {'label': 2}, 3: {'label': 10}, 4: {'label': 4}, 5: {'label': 20}, 6: {'label': 19}, 7: {'label': 17}, 8: {'label': 11}, 9: {'label': 6}, 10: {'label': 11}, 11: {'label': 11}, 12: {'label': 13}, 13: {'label': 7}, 14: {'label': 4}, 15: {'label': 4}, 16: {'label': 2}, 17: {'label': 11}, 18: {'label': 18}, 19: {'label': 17}, 20: {'label': 9}, 21: {'label': 11}, 22: {'label': 8}, 23: {'label': 6}, 24: {'label': 1}, 25: {'label': 7}, 26: {'label': 17}, 27: {'label': 1}, 28: {'label': 15}, 29: {'label': 16}, 30: {'label': 16}, 31: {'label': 8}, 32: {'label': 2}, 33: {'label': 16}, 34: {'label': 6}, 35: {'label': 6}, 36: {'label': 2}, 37: {'label': 7}, 38: {'label': 8}, 39: {'label': 11}, 40: {'label': 11}, 41: {'label': 15}, 42: {'label': 10}, 43: {'label': 11}, 44: {'label': 9}, 45: {'label': 16}, 46: {'label': 15}, 47: {'label': 4}, 48: {'label': 12}, 49: {'label': 7}, 50: {'label': 4}, 51: {'label': 16}, 52: {'label': 11}, 53: {'label': 6}, 54: {'label': 

In [1]:
# Compute all GNN-based metrics at once
from evaluation.evaluator import Evaluator
evaluator = Evaluator(device=device)
evaluator.evaluate_all(generated_dataset=grids, reference_dataset=lobsters)

NameError: name 'device' is not defined

In [13]:
# Alternatively, compute a single GNN-based metric. See evaluation/gin_evaluation.py for other metrics.
from evaluation.gin_evaluation import load_feature_extractor, MMDEvaluation

# Can tweak GIN hyperparameters, however defaults are set to our recommendations
gin = load_feature_extractor(device=device)
# Can tweak hyperparameters of MMD RBF, however defaults are set to our recommendations
mmd_eval = MMDEvaluation(gin)
result, time = mmd_eval.evaluate(generated_dataset=grids, reference_dataset=lobsters)
print('result: {}, time to compute: {:.3f}s'.format(result, time))

orthogonal
result: {'mmd_rbf': 1.3096486032009125}, time to compute: 0.084s


In [14]:
# Evaluation incorporating custom node/edge features.
# If no node features are provided, the default is degree features
# expressed as an integer.

node_feat_loc = 'feat'
edge_feat_loc = 'feat'
node_feat_dim = 10
edge_feat_dim = 5

# Create random node/edge features
for g in grids:
    num_nodes = g.number_of_nodes()
    g.ndata[node_feat_loc] = torch.randn(num_nodes, node_feat_dim).to(g.device)
    num_edges = g.number_of_edges()
    g.edata[edge_feat_loc] = torch.randn(num_edges, edge_feat_dim).to(g.device)
    
for g in lobsters:
    num_nodes = g.number_of_nodes()
    g.ndata[node_feat_loc] = torch.randn(num_nodes, node_feat_dim).to(g.device)
    num_edges = g.number_of_edges()
    g.edata[edge_feat_loc] = torch.randn(num_edges, edge_feat_dim).to(g.device)

# Load GIN
gin = load_feature_extractor(
    device=device, input_dim=node_feat_dim, edge_feat_dim=edge_feat_dim,
    node_feat_loc=node_feat_loc, edge_feat_loc=edge_feat_loc)

mmd_eval = MMDEvaluation(gin)
result, time = mmd_eval.evaluate(generated_dataset=lobsters, reference_dataset=grids)
print('result: {}, time to compute: {:.3f}s'.format(result, time))

orthogonal
result: {'mmd_rbf': 0.9569450616836548}, time to compute: 0.100s


In [15]:
# To quickly demonstrate sensitivity to node/edge features, let's create
# a copy of the grid graphs and slightly alter the node/edge feat distributions
import copy

result, time = mmd_eval.evaluate(generated_dataset=grids, reference_dataset=grids)
print('MMD RBF for grids with itself: {}'.format(result['mmd_rbf']))

grids_copy = [copy.deepcopy(g) for g in grids]
for g in grids_copy:
    g.ndata[node_feat_loc] *= 1.15
    g.edata[edge_feat_loc] *= 1.15
    
result, time = mmd_eval.evaluate(reference_dataset=grids, generated_dataset=grids_copy)
print('MMD RBF after altering node/edge feat. distributions: {}'.format(result['mmd_rbf']))

MMD RBF for grids with itself: 0
MMD RBF after altering node/edge feat. distributions: 0.03214991092681885
